In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dask.dataframe as dd
import pyspark
import csv
from datetime import date, datetime, timedelta
from itertools import groupby
from operator import itemgetter
import geopandas as gpd
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.mixture import GaussianMixture
import pyspark
from pyspark import SparkConf
import warnings
warnings.filterwarnings("ignore")

In [2]:
# sc.stop()

In [3]:
sc = pyspark.SparkContext()

In [4]:
crash_df = pd.read_csv('./data/Motor_Vehicle_Collisions_-_Crashes.csv',dtype={'BOROUGH':str,'ZIP CODE':str})

In [5]:
permit_df = pd.read_csv('./data/Street_Construction_Permits.csv',low_memory=False)

### Crash Dataset

In [6]:
crash_df['CRASH DATE'] = pd.to_datetime(crash_df['CRASH DATE'])
crash_df['CRASH DATE'].sort_values(ascending=False)
crash_df['dow'] = pd.to_datetime(crash_df['CRASH DATE']).dt.dayofweek
crash_df['year'] = pd.to_datetime(crash_df['CRASH DATE']).dt.year
crash_df['month'] = pd.to_datetime(crash_df['CRASH DATE']).dt.month
crash_df['day'] = pd.to_datetime(crash_df['CRASH DATE']).dt.day

In [7]:
crash = crash_df[crash_df['LATITUDE'].notna()].copy()
# crash['ON STREET NAME'].fillna(crash['CROSS STREET NAME'], inplace=True)
# crash['ON STREET NAME'].fillna(crash['OFF STREET NAME'], inplace=True)

In [8]:
crash = crash[(crash['ON STREET NAME'].notna())&(crash['NUMBER OF PERSONS INJURED'].notna())&(crash['NUMBER OF PEDESTRIANS KILLED'].notna())]
crash['ON STREET NAME'] = crash['ON STREET NAME'].apply(lambda x: x.strip())
crash['ON STREET NAME'] = crash['ON STREET NAME'].apply(lambda x: x.replace(' ',''))

In [9]:
geo_df = gpd.read_file('https://data.cityofnewyork.us/api/geospatial/d3c5-ddgc?method=export&format=Shapefile')

In [10]:
gdf_crash = gpd.GeoDataFrame(crash,crs='EPSG:4326',geometry=gpd.points_from_xy(crash.LONGITUDE, crash.LATITUDE))
crash_temp = gpd.sjoin(gdf_crash,geo_df,how='left',op='intersects')

In [11]:
crash_temp['BOROUGH'].fillna(crash_temp.borough,inplace=True)
crash_temp = crash_temp[crash_temp.BOROUGH.notna()].copy()

In [12]:
crash_final = crash_temp.copy()
crash_final['BOROUGH'] = crash_final['BOROUGH'].str.upper()
crash_final['key'] = list(zip(crash_final['BOROUGH'],crash_final['ON STREET NAME']))

In [13]:
crash_final.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,day,geometry,index_right,borough,location_i,objectid,shape_area,shape_leng,zone,key
0,2020-03-13,11:00,BROOKLYN,NaN,40.655518,-74.00644,"(40.655518, -74.00644)",3AVENUE,NaN,NaN,...,13,POINT (-74.00644 40.65552),227.0,Brooklyn,228.0,228.0,0.000993,0.177685,Sunset Park West,"(BROOKLYN, 3AVENUE)"
1,2020-05-19,12:04,BROOKLYN,NaN,40.603070,-74.01667,"(40.60307, -74.01667)",BELTPARKWAY,NaN,NaN,...,19,POINT (-74.01667 40.60307),11.0,Brooklyn,11.0,11.0,0.000265,0.079211,Bath Beach,"(BROOKLYN, BELTPARKWAY)"
3,2020-05-26,0:53,STATEN ISLAND,10306,40.574898,-74.09110,"(40.574898, -74.0911)",OLYMPIABOULEVARD,HUNTER AVENUE,NaN,...,26,POINT (-74.09110 40.57490),172.0,Staten Island,172.0,172.0,0.000658,0.118476,New Dorp/Midland Beach,"(STATEN ISLAND, OLYMPIABOULEVARD)"
4,2020-03-14,15:30,BROOKLYN,11234,40.625633,-73.90917,"(40.625633, -73.90917)",BERGENAVENUE,ROYCE PLACE,NaN,...,14,POINT (-73.90917 40.62563),154.0,Brooklyn,155.0,155.0,0.000868,0.169808,Marine Park/Mill Basin,"(BROOKLYN, BERGENAVENUE)"
6,2020-05-29,18:00,MANHATTAN,NaN,40.762410,-73.95443,"(40.76241, -73.95443)",FDRDRIVE,NaN,NaN,...,29,POINT (-73.95443 40.76241),138.0,Manhattan,140.0,140.0,0.000114,0.047584,Lenox Hill East,"(MANHATTAN, FDRDRIVE)"


### Permit Dataset

In [14]:
permit = permit_df[(permit_df['BoroughName'].notna()) & (permit_df['OnStreetName'].notna())\
                & (permit_df['IssuedWorkStartDate'].notna()) & (permit_df['IssuedWorkEndDate'].notna())].copy()
permit = permit[['PermitNumber','BoroughName','OnStreetName','IssuedWorkStartDate','IssuedWorkEndDate']].reset_index(drop=True)

In [15]:
permit.head(1)

,PermitNumber,BoroughName,OnStreetName,IssuedWorkStartDate,IssuedWorkEndDate
0,Q012007305130,QUEENS,CLOVERDALE BOULEVARD,11/01/2007 12:00:00 AM +0000,12/31/2007 12:00:00 AM +0000


### Check Street Names

In [16]:
permit_st = list(set(permit['OnStreetName'].to_list()))

In [17]:
crash_st = list(set(crash_final['ON STREET NAME'].to_list()))

In [18]:
crash_st

['',
 'PILGRIMAVENUE',
 'TIEMANNAVENUE',
 "SAINTMARY'SSTREET",
 'HOLLANDAVENUE',
 'METROPOLITANOVAL',
 'CAYUGAAVENUE',
 'LafayetteSt',
 'JAKECOURT',
 'LOCUSTPLACE',
 'TENTHSTREET',
 'FULLERCOURT',
 'SOMMERSLANE',
 'ELLINGTONPARKWAY',
 'ISABELLAAVENUE',
 'NOELROAD',
 'SPENCERAVENUE',
 '58LANE',
 'MUNDYAVENUE',
 'ROMANAVENUE',
 'EAST182STREET',
 'RogersAve',
 'EAST221STREET',
 '149DRIVE',
 '119ROAD',
 'GLENSTREET',
 'SUYDAMPLACE',
 'TIERSTREET',
 'BAYSHOREAVENUE',
 'BRIDGEPLAZACOURT',
 'AVENUEY',
 'HENDRICKSONSTREET',
 'RESERVIOROVALWEST',
 'OTSEGOSTREET',
 'WRIGHTAVENUE',
 'WILLIAMSBURGBRIDGE',
 'WILLMOHRSTREET',
 '29STREET',
 'ITTNERPLACE',
 'PLOUGHMANSBUSH',
 'BXPARKEAST',
 'MONSIGNORHALPINPLACE',
 'SHEARoad',
 'BEACH129STREET',
 'HANOVERAVENUE',
 'GROTONSTREET',
 'EasternPkwy',
 'ASTORIABLVDSOUTH',
 'LATHAMLANE',
 'JEWELAVENUE',
 '45-9231stStstreet',
 '49LANE',
 'HEYSONROAD',
 'GRACECOURT',
 '94STREET',
 'WEST155STREET',
 'SUNNYSIDECOURT',
 '24DRIVE',
 'BEACH142STREET',
 '208STREET',

In [19]:
'Jamaica' in [i[:7] for i in permit_st]

False

In [20]:
ps_space = [i.replace(' ','') for i in permit_st]
cs_space = [i.replace(' ','') for i in crash_st]

In [21]:
cs_space

['',
 'PILGRIMAVENUE',
 'TIEMANNAVENUE',
 "SAINTMARY'SSTREET",
 'HOLLANDAVENUE',
 'METROPOLITANOVAL',
 'CAYUGAAVENUE',
 'LafayetteSt',
 'JAKECOURT',
 'LOCUSTPLACE',
 'TENTHSTREET',
 'FULLERCOURT',
 'SOMMERSLANE',
 'ELLINGTONPARKWAY',
 'ISABELLAAVENUE',
 'NOELROAD',
 'SPENCERAVENUE',
 '58LANE',
 'MUNDYAVENUE',
 'ROMANAVENUE',
 'EAST182STREET',
 'RogersAve',
 'EAST221STREET',
 '149DRIVE',
 '119ROAD',
 'GLENSTREET',
 'SUYDAMPLACE',
 'TIERSTREET',
 'BAYSHOREAVENUE',
 'BRIDGEPLAZACOURT',
 'AVENUEY',
 'HENDRICKSONSTREET',
 'RESERVIOROVALWEST',
 'OTSEGOSTREET',
 'WRIGHTAVENUE',
 'WILLIAMSBURGBRIDGE',
 'WILLMOHRSTREET',
 '29STREET',
 'ITTNERPLACE',
 'PLOUGHMANSBUSH',
 'BXPARKEAST',
 'MONSIGNORHALPINPLACE',
 'SHEARoad',
 'BEACH129STREET',
 'HANOVERAVENUE',
 'GROTONSTREET',
 'EasternPkwy',
 'ASTORIABLVDSOUTH',
 'LATHAMLANE',
 'JEWELAVENUE',
 '45-9231stStstreet',
 '49LANE',
 'HEYSONROAD',
 'GRACECOURT',
 '94STREET',
 'WEST155STREET',
 'SUNNYSIDECOURT',
 '24DRIVE',
 'BEACH142STREET',
 '208STREET',

In [22]:
len([i for i in cs_space if i not in ps_space])

1127

In [23]:
# [i for i in cs_space if i not in ps_space]

In [24]:
permit['OnStreetName'] = permit['OnStreetName'].apply(lambda x: x.replace(' ',''))

### Permit Spark Process

In [25]:
permit.to_csv('permit.csv',index=False)

In [26]:
def data_time_range(record):
    import datetime
    split_sdate = record[3].split(' ')[0].split('/')
    start = (split_sdate[2],split_sdate[0],split_sdate[1])
    start = datetime.datetime(*map(int, start ))
    
    split_edate = record[4].split(' ')[0].split('/')
    end = (split_edate[2],split_edate[0],split_edate[1])
    end = datetime.datetime(*map(int, end ))
    return (record[1],record[2],start,end)
def year_filter(record):
    import datetime
    sd = datetime.datetime(2016,1,1)
    ed = datetime.datetime(2019,12,31)
    if record[2] > sd and record[3] < ed:
        return True
    else:
        return False
def range_of_date(record):
    from datetime import date, datetime, timedelta
    l = []
    for result in perdelta(record[2], record[3], timedelta(days=1)):
        l.append((result,1))
    res_tuple = []
    for i in l:
        res_tuple.append(i)
    return ((record[0],record[1]),res_tuple)

def perdelta(start, end, delta):
    curr = start
    while curr <= end:
        yield curr
        curr += delta

# def date_count(record):
#     import pyspark
#     spark = pyspark.SparkContext()
#     date_list = spark.parallelize(record[1]).reduceByKey(lambda x,y: x+y).collect()
#     return (record[0],date_list)

def date_count(record):
    d_list = record[1]
    first = itemgetter(0)
    sums = [(k, sum(item[1] for item in tups_to_sum))
        for k, tups_to_sum in groupby(sorted(d_list, key=first), key=first)]
    return [record[0],sums]

def output(record):
    return [record[0][0],record[0][1],record[1]]

final_list = sc.textFile('permit.csv')\
    .map(lambda x: next(csv.reader([x])))\
    .filter(lambda x: x[0] != 'PermitNumber')\
    .map(data_time_range)\
    .filter(year_filter)\
    .map(range_of_date)\
    .reduceByKey(lambda x,y: x+y)\
    .map(date_count)\
    .map(output)\
    .collect()

In [27]:
# range of dates from 2016-1-1 to 2019-12-31
date_col = []
for result in perdelta(datetime(2016,1,1),datetime(2019,12,31), timedelta(days=1)):
        date_col.append(result)
len(date_col)

1461

In [28]:
# create and finalize the dataframe
final_list = np.array(final_list,dtype=object)
permit_df_final = pd.DataFrame(final_list[:,:2],columns=['borough','street'])
for col in date_col:
    permit_df_final[col] = 0
dates = final_list[:,2]
for i, row in permit_df_final.iterrows():
    range_date = dates[i]
    for a in range_date:
        permit_df_final.at[i,a[0]]=a[1]
permit_df_final['key']=list(zip(permit_df_final['borough'],permit_df_final['street']))
first_column = permit_df_final.pop('key')
permit_df_final.insert(0, 'key', first_column)

In [29]:
# count total number of constructions for each street
permit_final = permit_df_final.copy()
permit_final['total_constructions']=permit_final.iloc[:,3:].sum(axis=1)
column = permit_final.pop('total_constructions')
permit_final.insert(3, 'total_constructions', column)
permit_final

,key,borough,street,total_constructions,2016-01-01 00:00:00,2016-01-02 00:00:00,2016-01-03 00:00:00,2016-01-04 00:00:00,2016-01-05 00:00:00,2016-01-06 00:00:00,...,2019-12-22 00:00:00,2019-12-23 00:00:00,2019-12-24 00:00:00,2019-12-25 00:00:00,2019-12-26 00:00:00,2019-12-27 00:00:00,2019-12-28 00:00:00,2019-12-29 00:00:00,2019-12-30 00:00:00,2019-12-31 00:00:00
0,"(QUEENS, 62STREET)",QUEENS,62STREET,21785,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,"(MANHATTAN, WEST144STREET)",MANHATTAN,WEST144STREET,17562,0,0,0,0,0,0,...,1,1,2,2,2,2,2,0,0,0
2,"(BRONX, WEST236STREET)",BRONX,WEST236STREET,21688,0,0,0,0,0,0,...,9,9,9,9,9,9,0,0,0,0
3,"(STATEN ISLAND, GIRARDSTREET)",STATEN ISLAND,GIRARDSTREET,111,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"(QUEENS, CARLTONPLACE)",QUEENS,CARLTONPLACE,947,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9346,"(MANHATTAN, WEST180STREET)",MANHATTAN,WEST180STREET,14228,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
9347,"(STATEN ISLAND, FLORIDAAVENUE)",STATEN ISLAND,FLORIDAAVENUE,1342,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9348,"(BROOKLYN, SHERLOCKPLACE)",BROOKLYN,SHERLOCKPLACE,227,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9349,"(QUEENS, ELDERTSLANE)",QUEENS,ELDERTSLANE,1067,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Mapping

In [30]:
permit_key = permit_final.key.tolist()

In [31]:
crash_final.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,day,geometry,index_right,borough,location_i,objectid,shape_area,shape_leng,zone,key
0,2020-03-13,11:00,BROOKLYN,NaN,40.655518,-74.00644,"(40.655518, -74.00644)",3AVENUE,NaN,NaN,...,13,POINT (-74.00644 40.65552),227.0,Brooklyn,228.0,228.0,0.000993,0.177685,Sunset Park West,"(BROOKLYN, 3AVENUE)"
1,2020-05-19,12:04,BROOKLYN,NaN,40.603070,-74.01667,"(40.60307, -74.01667)",BELTPARKWAY,NaN,NaN,...,19,POINT (-74.01667 40.60307),11.0,Brooklyn,11.0,11.0,0.000265,0.079211,Bath Beach,"(BROOKLYN, BELTPARKWAY)"
3,2020-05-26,0:53,STATEN ISLAND,10306,40.574898,-74.09110,"(40.574898, -74.0911)",OLYMPIABOULEVARD,HUNTER AVENUE,NaN,...,26,POINT (-74.09110 40.57490),172.0,Staten Island,172.0,172.0,0.000658,0.118476,New Dorp/Midland Beach,"(STATEN ISLAND, OLYMPIABOULEVARD)"
4,2020-03-14,15:30,BROOKLYN,11234,40.625633,-73.90917,"(40.625633, -73.90917)",BERGENAVENUE,ROYCE PLACE,NaN,...,14,POINT (-73.90917 40.62563),154.0,Brooklyn,155.0,155.0,0.000868,0.169808,Marine Park/Mill Basin,"(BROOKLYN, BERGENAVENUE)"
6,2020-05-29,18:00,MANHATTAN,NaN,40.762410,-73.95443,"(40.76241, -73.95443)",FDRDRIVE,NaN,NaN,...,29,POINT (-73.95443 40.76241),138.0,Manhattan,140.0,140.0,0.000114,0.047584,Lenox Hill East,"(MANHATTAN, FDRDRIVE)"


In [32]:
crash_permit = crash_final[crash_final.key.isin(permit_key)]
crash_permit.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,day,geometry,index_right,borough,location_i,objectid,shape_area,shape_leng,zone,key
0,2020-03-13,11:00,BROOKLYN,NaN,40.655518,-74.00644,"(40.655518, -74.00644)",3AVENUE,NaN,NaN,...,13,POINT (-74.00644 40.65552),227.0,Brooklyn,228.0,228.0,0.000993,0.177685,Sunset Park West,"(BROOKLYN, 3AVENUE)"
1,2020-05-19,12:04,BROOKLYN,NaN,40.603070,-74.01667,"(40.60307, -74.01667)",BELTPARKWAY,NaN,NaN,...,19,POINT (-74.01667 40.60307),11.0,Brooklyn,11.0,11.0,0.000265,0.079211,Bath Beach,"(BROOKLYN, BELTPARKWAY)"
3,2020-05-26,0:53,STATEN ISLAND,10306,40.574898,-74.09110,"(40.574898, -74.0911)",OLYMPIABOULEVARD,HUNTER AVENUE,NaN,...,26,POINT (-74.09110 40.57490),172.0,Staten Island,172.0,172.0,0.000658,0.118476,New Dorp/Midland Beach,"(STATEN ISLAND, OLYMPIABOULEVARD)"
4,2020-03-14,15:30,BROOKLYN,11234,40.625633,-73.90917,"(40.625633, -73.90917)",BERGENAVENUE,ROYCE PLACE,NaN,...,14,POINT (-73.90917 40.62563),154.0,Brooklyn,155.0,155.0,0.000868,0.169808,Marine Park/Mill Basin,"(BROOKLYN, BERGENAVENUE)"
6,2020-05-29,18:00,MANHATTAN,NaN,40.762410,-73.95443,"(40.76241, -73.95443)",FDRDRIVE,NaN,NaN,...,29,POINT (-73.95443 40.76241),138.0,Manhattan,140.0,140.0,0.000114,0.047584,Lenox Hill East,"(MANHATTAN, FDRDRIVE)"


In [33]:
crash_permit.isna().sum()

CRASH DATE                             0
CRASH TIME                             0
BOROUGH                                0
ZIP CODE                          249843
LATITUDE                               0
LONGITUDE                              0
LOCATION                               0
ON STREET NAME                         0
CROSS STREET NAME                 201464
OFF STREET NAME                  1190721
NUMBER OF PERSONS INJURED              0
NUMBER OF PERSONS KILLED               9
NUMBER OF PEDESTRIANS INJURED          0
NUMBER OF PEDESTRIANS KILLED           0
NUMBER OF CYCLIST INJURED              0
NUMBER OF CYCLIST KILLED               0
NUMBER OF MOTORIST INJURED             0
NUMBER OF MOTORIST KILLED              0
CONTRIBUTING FACTOR VEHICLE 1       3330
CONTRIBUTING FACTOR VEHICLE 2     159946
CONTRIBUTING FACTOR VEHICLE 3    1116950
CONTRIBUTING FACTOR VEHICLE 4    1174897
CONTRIBUTING FACTOR VEHICLE 5    1186526
COLLISION_ID                           0
VEHICLE TYPE COD

In [34]:
save = ['key','CRASH DATE','CRASH TIME','BOROUGH','LATITUDE','LONGITUDE','ON STREET NAME',
 'NUMBER OF PERSONS INJURED','NUMBER OF PERSONS KILLED','NUMBER OF PEDESTRIANS INJURED',
 'NUMBER OF PEDESTRIANS KILLED','NUMBER OF CYCLIST INJURED','NUMBER OF CYCLIST KILLED',
 'NUMBER OF MOTORIST INJURED','NUMBER OF MOTORIST KILLED','dow','year','month','day','geometry',]

In [35]:
cp_df = crash_permit[save].copy()
cp_df.dropna(inplace=True)
cp_df['construction'] = 0
cp_df.head(1)

,key,CRASH DATE,CRASH TIME,BOROUGH,LATITUDE,LONGITUDE,ON STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,...,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,dow,year,month,day,geometry,construction
0,"(BROOKLYN, 3AVENUE)",2020-03-13,11:00,BROOKLYN,40.655518,-74.00644,3AVENUE,0.0,0.0,0,...,0,0,0,0,4,2020,3,13,POINT (-74.00644 40.65552),0


In [36]:
cp_df = cp_df[(cp_df['CRASH DATE'] < datetime(2020,1,1)) & (cp_df['CRASH DATE'] > datetime(2015,12,31))]

In [37]:
for index, row in cp_df.iterrows():
    key = row['key']
    date = row['CRASH DATE']
    value = permit_final.loc[permit_final['key'] == key][date].copy()
    cp_df.at[index,'construction'] = value

In [38]:
cp_df.head()

,key,CRASH DATE,CRASH TIME,BOROUGH,LATITUDE,LONGITUDE,ON STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,...,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,dow,year,month,day,geometry,construction
107,"(BRONX, MAJORDEEGANEXPRESSWAY)",2018-11-22,15:15,BRONX,40.875122,-73.905174,MAJORDEEGANEXPRESSWAY,1.0,0.0,0,...,0,0,1,0,3,2018,11,22,POINT (-73.90517 40.87512),1
585,"(BRONX, MAJORDEEGANEXPRESSWAY)",2018-11-23,6:30,BRONX,40.839600,-73.929474,MAJORDEEGANEXPRESSWAY,0.0,0.0,0,...,0,0,0,0,4,2018,11,23,POINT (-73.92947 40.83960),1
613,"(QUEENS, 45STREET)",2018-08-18,3:15,QUEENS,40.743263,-73.919640,45STREET,0.0,0.0,0,...,0,0,0,0,5,2018,8,18,POINT (-73.91964 40.74326),24
777,"(BROOKLYN, BELTPARKWAY)",2018-09-06,17:10,BROOKLYN,40.651863,-73.865360,BELTPARKWAY,0.0,0.0,0,...,0,0,0,0,3,2018,9,6,POINT (-73.86536 40.65186),2
1780,"(BROOKLYN, NOSTRANDAVENUE)",2018-07-31,11:45,BROOKLYN,40.633995,-73.947780,NOSTRANDAVENUE,1.0,0.0,1,...,0,0,0,0,1,2018,7,31,POINT (-73.94778 40.63399),132


In [41]:
cp_df.to_csv('new_cp_data.csv',index=False)

In [42]:
cp_df.shape

(573096, 21)

In [54]:
cp_agg_accident = cp_df.copy()

In [55]:
cp_agg_accident.columns.tolist()

['key',
 'CRASH DATE',
 'CRASH TIME',
 'BOROUGH',
 'LATITUDE',
 'LONGITUDE',
 'ON STREET NAME',
 'NUMBER OF PERSONS INJURED',
 'NUMBER OF PERSONS KILLED',
 'NUMBER OF PEDESTRIANS INJURED',
 'NUMBER OF PEDESTRIANS KILLED',
 'NUMBER OF CYCLIST INJURED',
 'NUMBER OF CYCLIST KILLED',
 'NUMBER OF MOTORIST INJURED',
 'NUMBER OF MOTORIST KILLED',
 'dow',
 'year',
 'month',
 'day',
 'geometry',
 'construction']

In [56]:
grouby_col = ['key','CRASH DATE','CRASH TIME','BOROUGH','LATITUDE','LONGITUDE','ON STREET NAME','dow',
 'year','month','day','geometry','construction']
sum_col = ['NUMBER OF PERSONS INJURED',
 'NUMBER OF PERSONS KILLED',
 'NUMBER OF PEDESTRIANS INJURED',
 'NUMBER OF PEDESTRIANS KILLED',
 'NUMBER OF CYCLIST INJURED',
 'NUMBER OF CYCLIST KILLED',
 'NUMBER OF MOTORIST INJURED',
 'NUMBER OF MOTORIST KILLED']
sum_by_row = 'sum'

In [57]:
agg_dict = {row:sum_by_row for row in sum_col}
agg_dict['key'] = 'count'
agg_dict

{'NUMBER OF PERSONS INJURED': 'sum',
 'NUMBER OF PERSONS KILLED': 'sum',
 'NUMBER OF PEDESTRIANS INJURED': 'sum',
 'NUMBER OF PEDESTRIANS KILLED': 'sum',
 'NUMBER OF CYCLIST INJURED': 'sum',
 'NUMBER OF CYCLIST KILLED': 'sum',
 'NUMBER OF MOTORIST INJURED': 'sum',
 'NUMBER OF MOTORIST KILLED': 'sum',
 'key': 'count'}

In [58]:
sel_col = ['key','CRASH DATE','CRASH TIME','BOROUGH','LATITUDE','LONGITUDE',
           'ON STREET NAME','construction','dow',
 'year','month','day','NUMBER OF PERSONS INJURED',
 'NUMBER OF PERSONS KILLED',
 'NUMBER OF PEDESTRIANS INJURED',
 'NUMBER OF PEDESTRIANS KILLED',
 'NUMBER OF CYCLIST INJURED',
 'NUMBER OF CYCLIST KILLED',
 'NUMBER OF MOTORIST INJURED',
 'NUMBER OF MOTORIST KILLED']

In [59]:
cp_agg_accident = cp_agg_accident[sel_col].groupby(['CRASH DATE','CRASH TIME','BOROUGH','LATITUDE','LONGITUDE','ON STREET NAME','dow',
 'year','month','day','construction']).agg(agg_dict).reset_index().rename(columns={'key':'Number of Accident'})

In [60]:
cp_agg_accident.head()

,CRASH DATE,CRASH TIME,BOROUGH,LATITUDE,LONGITUDE,ON STREET NAME,dow,year,month,day,construction,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,Number of Accident
0,2016-01-01,0:01,MANHATTAN,40.760223,-73.967462,EAST57STREET,4,2016,1,1,0,0.0,0.0,0,0,0,0,0,0,1
1,2016-01-01,0:01,MANHATTAN,40.805057,-73.939034,EAST125STREET,4,2016,1,1,0,1.0,0.0,1,0,0,0,0,0,1
2,2016-01-01,0:05,MANHATTAN,40.799258,-73.939050,3AVENUE,4,2016,1,1,0,2.0,0.0,0,0,0,0,2,0,1
3,2016-01-01,0:15,MANHATTAN,40.738552,-73.999682,7AVENUE,4,2016,1,1,0,0.0,0.0,0,0,0,0,0,0,1
4,2016-01-01,0:23,BROOKLYN,40.653419,-73.888970,GEORGIAAVENUE,4,2016,1,1,0,0.0,0.0,0,0,0,0,0,0,1


In [61]:
cp_agg_accident.to_csv('cp_agg.csv',index=False)

In [ ]:
X = cp_agg_accident.iloc[:,6:]
X

In [ ]:
cluster_range = range(2,5)
for n_c in cluster_range:
    silhouette = []
    for i in range(1):
        km = KMeans(random_state=i*i, n_clusters=n_c, n_jobs=-1)
        cluster_labels = km.fit(X).labels_
        silhouette.append(silhouette_score(X, cluster_labels))
    silhouette_avg = np.mean(silhouette)
    print("For n_clusters = {},".format(n_c)+" the average silhouette_score is :{}".format(silhouette_avg))